In [4]:
#Create a envronment with ppthon 3.9

In [5]:

# !pip install spacy_transformers
# !pip install -U pip setuptools wheel
# !pip install -U 'spacy[apple]'

In [6]:
import spacy
spacy.__version__



'3.7.2'

In [7]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm
import sklearn
import json

In [12]:
resume_training_data = json.load(open('/Users/amarnathgowda/Desktop/ML_Project/ResumeParsing/SpacyProject/training/ResumeJsondataV1.json')) 


To Create the config file using the base config file

You can execute in notebook or in terminal, you will get the config file in the desire folder

!python -m spacy init fill-config /Users/amarnathgowda/Desktop/ML_Project/ResumeParsing/SpacyProject/spacyConfig/base_config.cfg /Users/amarnathgowda/Desktop/ML_Project/ResumeParsing/SpacyProject/spacyConfig/config.cfg

In [ ]:
# !python -m spacy init fill-config /Users/amarnathgowda/Desktop/ML_Project/ResumeParsing/SpacyProject/spacyConfig/base_config.cfg /Users/amarnathgowda/Desktop/ML_Project/ResumeParsing/SpacyProject/spacyConfig/config.cfg

To check the wheter the data annotetion is correcte, will check two entity annotetion didnt got mearched each othere in the single resume

In [14]:
def get_spacy_doc(file,data):
  nlp = spacy.blank('en')
  db = DocBin()

  for text, annot in tqdm(data):
    doc = nlp.make_doc(text)
    annot = annot['entities']

    ents =[]
    entity_indices = []

    for start, end, label in annot:
      skip_entity=False
      for idx in range(start, end):
        if idx in entity_indices:
          skip_entity=True
          break
      if skip_entity==True:
        continue
      entity_indices = entity_indices+list(range(start,end))

      try:
        span = doc.char_span(start, end, label=label, alignment_mode='strict')
      except:
        continue

      if span is None:
        err_data = str([start, end]) + " "+str(text)+"\n"
        file.write(err_data)

      else:
        ents.append(span)

    try:
      doc.ents = ents
      db.add(doc)
    except:
      pass

  return db


In [18]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(resume_training_data, test_size=0.3)

In [19]:
len(train), len(test)

(21, 10)

In [20]:
file = open('/Users/amarnathgowda/Desktop/ML_Project/ResumeParsing/SpacyProject/spacyTrainTestdata/error.txt', 'w')

db = get_spacy_doc(file, train)
db.to_disk('/Users/amarnathgowda/Desktop/ML_Project/ResumeParsing/SpacyProject/spacyTrainTestdata/train_data.spacy')

db = get_spacy_doc(file, test)
db.to_disk('/Users/amarnathgowda/Desktop/ML_Project/ResumeParsing/SpacyProject/spacyTrainTestdata/test_data.spacy')

file.close()

/Users/amarnathgowda/anaconda3/envs/newenvr/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|██████████| 10/10 [00:00<00:00, 90.21it/s]


In [21]:
len(db.tokens)

10

To train the model using our train and test data, below command can execute in the notebook or terminal

!python -m spacy train /Users/amarnathgowda/Desktop/ML_Project/ResumeParsing/SpacyProject/spacyConfig/config.cfg --output /Users/amarnathgowda/Desktop/ML_Project/ResumeParsing/SpacyProject/modeloutput --paths.train /Users/amarnathgowda/Desktop/ML_Project/ResumeParsing/SpacyProject/spacyTrainTestdata/train_data.spacy --paths.dev /Users/amarnathgowda/Desktop/ML_Project/ResumeParsing/SpacyProject/spacyTrainTestdata/test_data.spacy 

In [ ]:
# !python -m spacy train /Users/amarnathgowda/Desktop/ML_Project/ResumeParsing/SpacyProject/spacyConfig/config.cfg --output /Users/amarnathgowda/Desktop/ML_Project/ResumeParsing/SpacyProject/modeloutput --paths.train /Users/amarnathgowda/Desktop/ML_Project/ResumeParsing/SpacyProject/spacyTrainTestdata/train_data.spacy --paths.dev /Users/amarnathgowda/Desktop/ML_Project/ResumeParsing/SpacyProject/spacyTrainTestdata/test_data.spacy --gpu-id 0

In [22]:
nlp = spacy.load('/Users/amarnathgowda/Desktop/ML_Project/ResumeParsing/SpacyProject/modeloutput/model-best')

In [23]:
doc = nlp('my name is Laxmi Kant Tiwari. I worked at Microsoft. I have 10 years of experience')
for ent in doc.ents:
  print(ent.text, "   ->>>>> ", ent.label_)

Laxmi Kant    ->>>>>  NAME
Microsoft    ->>>>>  ORGANIZATION
10 years of experience    ->>>>>  EXPERIENCE


In [24]:
!pip install pypdf4

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pypdf4: filename=PyPDF4-1.27.0-py3-none-any.whl size=61226 sha256=268be18406993fe8123b6d4d41a45f23ac1e86f8ad3bd250d4a07baed5059714
  Stored in directory: /Users/amarnathgowda/Library/Caches/pip/wheels/cd/0e/4a/e6f842a6035ccffff0dab29c39dd06c3427560a82783355a83
Successfully built pypdf4
DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of textract or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [41]:
from PyPDF4 import PdfFileReader
pdf = PdfFileReader("/Users/amarnathgowda/Desktop/ML_Project/ResumeParsing/files/Alice Clark CV.pdf")
print(pdf.numPages)
text =''
for page in range(0,pdf.numPages):
    
    first_page = pdf.getPage(page)
    print(page)
    text = text+first_page.extractText()
    print(text)

2
0
Alice Clark
 
AI
 
/ Machine Learning
 
 
Delhi, India Em
ail me 
on Indeed
 

 
20+ y
ears of experience in data han
dling, design
, and 
development
 

 
Data Wa
rehou
se: 
Data analysis
, star
/snow fla
ke scema data 
modelling
 
and design specific to 
data warehousing
 
and business intelligence
 

 
Database: Experience in database designing, scalability, back
-
up and recovery, writing and
 
optimizing SQL code and Stored Procedures, creating functions, views, triggers and indexes.
 
Cloud platform: Worked on Microsoft Azure cloud services like Document 
DB, SQL Azure, 
Stream
 
Analytics, Event hub, Power BI, Web Job, Web App, Power BI, Azure data lake 
analytics(U
-
SQL)
 
Willing to relocate anywhere
 
 
W
ORK EXPERIE
NCE
 
Software Engineer
 
Microsoft 

 
Bangalore,
 
Karnataka
 
January
 
20
0
0 to Present
 
1. Microsoft Rewards Live dashboards:
 
Description: 
-
 
Microsoft rewards is loyalty program that rewards Users for browsing and shopping
 
online. Microsoft Rew

In [42]:
doc = nlp(text)
for ent in doc.ents:    
    print(ent.text, " ->>>>> ", ent.label_)

Machine Learning, 
Natural
 
Language
 
Processing, and 
Big Data Handling
 
 
ADDITIONAL INFO
RMATION
 
Professional Skills
 

 
skills with ability to interact with individuals at all the levels
 

ger and team members and
 
good performer both in team and independent job environments
 

 

 
  ->>>>>  SKILLS


In [46]:
import textract

text=textract.process("/Users/amarnathgowda/Desktop/ML_Project/ResumeParsing/files/SamanAyubi_4_6_.pdf")

print(text)

text = text.decode()
text = text.split('\n')
text = "".join(text)
print(text)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


b'Saman Ayubi\n\nFrontend Developer\nBeing motivated with good attitude and strong analytical and development skill to work with the most competent professionals leveraging my experience\nto upgrade my technical and management skills for a continued professional growth and advancement.\n\nayubisaman90@gmail.com\n\n8142316777\n\nBangalore\n\nlinkedin.com/in/saman-ayubi-64498992\n\nWORK EXPERIENCE\nAssociate Consultant\nCapgemini\n05/2018 \xe2\x80\x93 Present\n\nTeam Lead\nIntellect Design Arena\n05/2016 \xe2\x80\x93 05/2018\n\nConsultant\nIntellect Design Arena\n06/2015 \xe2\x80\x93 04/2016\n\nEDUCATION\nElectronics & Communication (B.tech)\nChandigarh Engineering College\n08/2011 \xe2\x80\x93 05/2015\n\nSKILLS\n\nAngular 2/4/5/7\n\nJavaScript\n\nHtml 5\n\nCss 3\n\nBanglore\n\nExt JS 5/6\n\nAgile\n\nDocker\n\nJenkins\n\nGit\n\nSVN\n\nAngular Material\n\nHighcharts\n\nGoogle Maps\n\nGoogle Analytics\n\nAmcharts\n\nPredix Ui Components\n\nHyderabad\n\nHyderabad\n\nMohali\n\nPROJECTS\n1. E

In [47]:
doc = nlp(text)
for ent in doc.ents:    
    print(ent.text, " ->>>>> ", ent.label_)


Saman AyubiFrontend  ->>>>>  NAME
(B.tech)Chandigarh Engineering College08/2011  ->>>>>  QUALIFICATION
